## Train XGB on TBI signal data

Code to train an XGBoost model on the TBI data to predict hypoxemia in the future (low SAO2).

Note that the data is private and we are unable to make it publicly available in this repo.

In [ ]:
from tbi_downstream_prediction import *
PATH = "/homes/gws/hughchen/phase/tbi_subset/"
DPATH = PATH+"tbi/processed_data/hypoxemia/"
RESULTPATH = PATH+"results/"

# Set important variables
label_type = "desat_bool92_5_nodesat"; eta = 0.02
hosp_data = "tbi"; data_type = "raw[top11]"
mod_type = "xgb_{}_eta{}".format(label_type,eta)

# Set up result directory
RESDIR = '{}results/{}/'.format(PATH, mod_type)
if not os.path.exists(RESDIR): os.makedirs(RESDIR)

# Load tbi data
y_tbi = np.load(DPATH+"tbiy.npy",mmap_mode="r")
X_tbi = np.load(DPATH+"X_tbi_imp_standard.npy",mmap_mode="r")

feats = ["ECGRATE", "ETCO2", "ETSEV", "ETSEVO", "FIO2", "NIBPD", "NIBPM", 
         "NIBPS","PEAK", "PEEP", "PIP", "RESPRATE", "SAO2", "TEMP1", "TV"]
weird_feats = ["ETSEV", "PIP", "PEEP", "TV"]
feat_inds = np.array([feats.index(feat) for feat in feats if feat not in weird_feats])    
X_tbi2 = X_tbi[:,feat_inds,:]
(X_test, y_test, X_valid, y_valid, X_train, y_train) = split_data(DPATH,X_tbi2,y_tbi)
feat_lst2 = [feat for feat in feats if feat not in weird_feats]

# Set parameters to train model
param = {'max_depth':6, 'eta':eta, 'subsample':0.5, 'gamma':1.0, 
         'min_child_weight':10, 'base_score':y_train.mean(), 
         'objective':'binary:logistic', 'eval_metric':["logloss"]}